In [ ]:
!pip install pynvml GPUtil

In [ ]:
import torch
import torch.nn.functional as F
import numpy as np
import cv2
import time
import os
from skimage.metrics import peak_signal_noise_ratio as psnr
import matplotlib.pyplot as plt

def gaussian_filter_gpu(image_tensor, ksize=5, sigma=1.0):
    """
    Apply Gaussian Filter on the input image using GPU.

    Parameters:
    - image_tensor: Input image tensor (binary or 3-channel)
    - ksize: Kernel size (odd integer)
    - sigma: Gaussian kernel standard deviation

    Returns:
    - filtered_image_tensor: Output image tensor after applying Gaussian filter
    """
    if ksize % 2 == 0:
        ksize += 1

    def gaussian_kernel(ksize, sigma, channels):
        kernel = np.fromfunction(
            lambda x, y: (1 / (2 * np.pi * sigma**2)) *
                         np.exp(-((x - (ksize - 1) / 2) ** 2 + (y - (ksize - 1) / 2) ** 2) / (2 * sigma ** 2)),
            (ksize, ksize)
        )
        kernel = kernel / kernel.sum()
        kernel = torch.tensor(kernel, dtype=torch.float32).unsqueeze(0).unsqueeze(0)
        return kernel.repeat(channels, 1, 1, 1).cuda()

    channels = image_tensor.size(0)
    kernel = gaussian_kernel(ksize, sigma, channels)
    padding = ksize // 2

    filtered_image_tensor = F.conv2d(image_tensor.unsqueeze(0).cuda(), kernel, padding=padding, groups=channels)

    return filtered_image_tensor.squeeze().cpu()

def measure_metrics(image, filtered_image, inference_time):

    gpu_name = torch.cuda.get_device_name(0)
    gpu_memory_allocated = torch.cuda.memory_allocated() / (1024 * 1024)
    gpu_memory_reserved = torch.cuda.memory_reserved() / (1024 * 1024)
    gpu_temperature = torch.cuda.temperature(0)
    gpu_load = torch.cuda.utilization(0)

    psnr_value = psnr(image, filtered_image)

    metrics = {
        "GPU": gpu_name,
        "GPU Memory Usage (MB)": f"{gpu_memory_allocated:.2f} MB / {gpu_memory_reserved:.2f} MB",
        "Temperature (°C)": gpu_temperature,
        "GPU Load (%)": gpu_load,
        "Inference Time (ms)": inference_time,
        "PSNR (dB)": psnr_value,
    }

    return metrics

def process_image(image_path, output_path, ksize=5, sigma=1.0):
    """
    Load image, apply Gaussian Filter, measure metrics, and save the result.

    Parameters:
    - image_path: Path to the input image
    - output_path: Path to save the filtered image
    - ksize: Kernel size (odd integer)
    - sigma: Gaussian kernel standard deviation
    """
    image = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)

    if image.shape[0] > 720 or image.shape[1] > 1280:
        raise ValueError("Image size exceeds the allowed dimensions of 1280x720p")

    image_tensor = torch.tensor(image, dtype=torch.float32).cuda()
    if len(image_tensor.shape) == 2:
        image_tensor = image_tensor.unsqueeze(0)
    else:
        image_tensor = image_tensor.permute(2, 0, 1)

    image_tensor = image_tensor / 255.0

    start_time = time.time()

    filtered_image_tensor = gaussian_filter_gpu(image_tensor, ksize=ksize, sigma=sigma)

    inference_time = (time.time() - start_time) * 1000  # in milliseconds

    filtered_image = filtered_image_tensor.cpu().numpy()
    if len(filtered_image.shape) == 3:
        filtered_image = np.moveaxis(filtered_image, 0, -1)
    else:
        filtered_image = filtered_image.squeeze()
    filtered_image = (filtered_image * 255).astype(np.uint8)

    metrics = measure_metrics(image, filtered_image, inference_time)

    cv2.imwrite(output_path, filtered_image)

    return metrics

def process_dataset(root_folder, ksize=5, sigma=1.0):
    """
    Process all images in the dataset and save filtered images to the respective folders.

    Parameters:
    - root_folder: Path to the root dataset folder
    - ksize: Kernel size (odd integer)
    - sigma: Gaussian kernel standard deviation
    """
    noisy_folders = ['noisy5', 'noisy15', 'noisy35']
    denoised_folders = ['denoised5', 'denoised15', 'denoised35']

    total_inference_time = 0
    total_psnr = 0
    image_count = 0

    start_total_time = time.time()

    for noisy_folder, denoised_folder in zip(noisy_folders, denoised_folders):
        input_folder = os.path.join(root_folder, noisy_folder)
        output_folder = os.path.join(root_folder, denoised_folder)

        if not os.path.exists(output_folder):
            os.makedirs(output_folder)

        for filename in os.listdir(input_folder):
            if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
                image_path = os.path.join(input_folder, filename)
                output_path = os.path.join(output_folder, filename)

                metrics = process_image(image_path, output_path, ksize, sigma)

                total_inference_time += metrics["Inference Time (ms)"]
                total_psnr += metrics["PSNR (dB)"]
                image_count += 1

    end_total_time = time.time()
    total_time_taken = (end_total_time - start_total_time) * 1000

    if image_count > 0:
        avg_inference_time = total_inference_time / image_count
        avg_psnr = total_psnr / image_count

        gpu_name = metrics["GPU"]
        gpu_memory_usage = metrics["GPU Memory Usage (MB)"]
        gpu_temperature = metrics["Temperature (°C)"]
        gpu_load = metrics["GPU Load (%)"]

        print(f"GPU Name: {gpu_name}")
        print(f"GPU Memory Usage: {gpu_memory_usage}")
        print(f"GPU Temperature (°C): {gpu_temperature}")
        print(f"GPU Load (%): {gpu_load}")
        print(f"Average Inference Time (ms): {avg_inference_time:.2f}")
        print(f"Average PSNR (dB): {avg_psnr:.2f}")
        print(f"Total Time Taken (ms): {total_time_taken:.2f}")
    else:
        print("No images processed.")

if __name__ == "__main__":

    root_folder = '/content/drive/MyDrive/dataset'
    process_dataset(root_folder, ksize=5, sigma=1.0)
